# Film Location Database Platform - Comprehensive Financial & Business Plan

This notebook provides detailed cost analysis, financial projections, and business planning for a **Film Industry Location Database Platform** using **Supabase** (backend) and **Vercel** (frontend hosting).

## Platform Overview
Service platform for film/TV production companies, advertising agencies, and location scouts to:
- Store and organize extensive location databases with high-resolution imagery
- Present and showcase filming locations to production companies and creative teams
- Enable freelancers and indie filmmakers to access curated location libraries
- Facilitate location scouting with searchable metadata, permits, and logistics info

## Target Customer Segments
1. **Production Companies** (Film/TV): $499-1,499/mo - Ongoing location management
2. **Advertising Agencies**: $999-2,999/mo - Multi-project access, high turnover
3. **Location Scouts/Freelancers**: $149-299/project - Single production access
4. **Streaming Platforms/Networks**: $2,999+/mo - Enterprise-scale libraries
5. **Independent Filmmakers**: $149/mo - Budget-conscious, smaller libraries

## Table of Contents
1. [Setup & Configuration](#setup)
2. [Cost Model & Assumptions](#cost-model)
3. [Interactive Business Calculator](#calculator)
4. [Revenue & Cost Projections](#projections)
5. [Scenario Analysis](#scenarios)
6. [Unit Economics & Metrics](#metrics)
7. [Break-Even Analysis](#breakeven)
8. [Strategic Recommendations](#recommendations)
9. [Export Business Plan](#export)

## 1. Setup & Configuration <a id="setup"></a>

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML
import warnings
warnings.filterwarnings('ignore')

# Set visualization styles
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Color palette
COLORS = {
    'primary': '#667eea',
    'secondary': '#764ba2',
    'success': '#28a745',
    'danger': '#dc3545',
    'warning': '#ffc107',
    'info': '#17a2b8'
}

print("✓ Libraries imported successfully")
print("✓ Visualization settings configured")

## 2. Cost Model & Assumptions <a id="cost-model"></a>

### Pricing Structure

#### Supabase Pro Plan
- **Base Cost**: $120/month
- **Storage**: 100 GB included, then $0.021/GB
- **Bandwidth**: 250 GB included, then $0.09/GB

#### Vercel Pro Plan
- **Base Cost**: $20/month per team seat
- **Bandwidth**: 1 TB (1000 GB) included, then $0.15/GB
- **Function Duration**: 40 hours/month included

In [ ]:
# Define pricing constants and business configurationclass PricingModel:    """Centralized pricing model and business assumptions for all calculations"""        # ===== Infrastructure Costs =====    # Supabase Pro Plan    SUPABASE_PRO_BASE = 120  # $/month base cost    SUPABASE_STORAGE_INCLUDED = 100  # GB included    SUPABASE_STORAGE_COST = 0.021  # $/GB overage    SUPABASE_BANDWIDTH_INCLUDED = 250  # GB included    SUPABASE_BANDWIDTH_COST = 0.09  # $/GB overage        # Vercel Pro Plan    VERCEL_PRO_PER_SEAT = 20  # $/month per seat    VERCEL_BANDWIDTH_INCLUDED = 1000  # GB (1 TB) included    VERCEL_BANDWIDTH_COST = 0.15  # $/GB overage        # ===== Usage-Based Pricing (AWS-style alternative) =====    USAGE_BASE_FEE = 99  # $/month platform access    USAGE_STORAGE_PER_GB = 0.08  # $/GB/month storage    USAGE_BANDWIDTH_PER_GB = 0.12  # $/GB bandwidth        # ===== Business Operating Costs =====    # Film industry specific defaults    DEFAULT_TEAM_COSTS = 15000  # $/month (developers, customer success, location curators)    DEFAULT_MARKETING_SPEND = 3000  # $/month (film festivals, trade shows, partnerships)    DEFAULT_OTHER_COSTS = 500  # $/month (legal, insurance, tools)    OTHER_FIXED_COSTS = DEFAULT_TEAM_COSTS + DEFAULT_MARKETING_SPEND + DEFAULT_OTHER_COSTS  # Total: 18500        # ===== Film Industry Pricing Tiers =====    TIER_FREELANCER = 149  # $/month    TIER_PROFESSIONAL = 499  # $/month    TIER_STUDIO = 1499  # $/month    TIER_ENTERPRISE = 3999  # $/month        # ===== SaaS Business Health Benchmarks =====    SAAS_GOOD_MARGIN = 20  # % net profit margin    SAAS_EXCELLENT_MARGIN = 50  # % net profit margin    SAAS_GOOD_GROSS_MARGIN = 70  # % gross margin    SAAS_GOOD_CHURN = 5  # % monthly churn    SAAS_EXCELLENT_CHURN = 3  # % monthly churn (film industry B2B)    SAAS_GOOD_LTV_CAC_RATIO = 3.0  # 3:1 or better    SAAS_MIN_LTV_CAC_RATIO = 1.0  # Below this is concerning        # ===== Edge Case Handling =====    # When churn is zero (infinite lifetime), cap at reasonable maximum    MAX_LIFETIME_MONTHS = 120  # 10 years (reasonable upper bound)        # When growth is zero, assume minimal growth for CAC calculation    MIN_GROWTH_FOR_CAC = 0.1  # 0.1% minimum to avoid division by zero        # When calculating ratios with zero denominators    RATIO_UNDEFINED_PLACEHOLDER = np.inf  # Use infinity, not arbitrary numberspricing = PricingModel()# Display pricing summarypricing_df = pd.DataFrame({    'Category': ['Supabase Base', 'Supabase Storage', 'Supabase Bandwidth',                  'Vercel Per Seat', 'Vercel Bandwidth',                 'Team Costs', 'Marketing', 'Other Operating'],    'Cost': ['$120/mo', '$0.021/GB (>100GB)', '$0.09/GB (>250GB)',              '$20/mo', '$0.15/GB (>1TB)',             f'${pricing.DEFAULT_TEAM_COSTS:,}/mo',              f'${pricing.DEFAULT_MARKETING_SPEND:,}/mo',             f'${pricing.DEFAULT_OTHER_COSTS}/mo'],    'Included/Notes': ['N/A', '100 GB', '250 GB', 'N/A', '1000 GB',                      'Developers, CS, curators', 'Festivals, trade shows', 'Legal, insurance']})display(HTML(pricing_df.to_html(index=False)))print(f"\n✓ Pricing model initialized")print(f"  Total default operating costs: ${pricing.OTHER_FIXED_COSTS:,}/month")

## 3. Interactive Business Calculator <a id="calculator"></a>

Use the sliders below to model your business and see real-time calculations.

In [ ]:
class BusinessCalculator:    """Core business calculation engine for film location database platform"""        def __init__(self, pricing_model):        self.pricing = pricing_model        def calculate_infrastructure_costs(self, customers, locations_per_customer,                                       images_per_location, avg_image_size_mb,                                      monthly_views_per_image, use_vercel=True,                                      vercel_team_members=1, vercel_bandwidth_multiplier=0.3):        """Calculate monthly infrastructure costs (Supabase + Vercel)"""                # Storage calculation        total_images = customers * locations_per_customer * images_per_location        total_storage_gb = (total_images * avg_image_size_mb) / 1024                # Bandwidth calculation        image_bandwidth_gb = (total_images * monthly_views_per_image * avg_image_size_mb) / 1024        frontend_bandwidth_gb = image_bandwidth_gb * vercel_bandwidth_multiplier if use_vercel else 0                # Supabase costs        storage_overage = max(0, total_storage_gb - self.pricing.SUPABASE_STORAGE_INCLUDED)        storage_cost = storage_overage * self.pricing.SUPABASE_STORAGE_COST                bandwidth_overage = max(0, image_bandwidth_gb - self.pricing.SUPABASE_BANDWIDTH_INCLUDED)        bandwidth_cost = bandwidth_overage * self.pricing.SUPABASE_BANDWIDTH_COST                supabase_total = self.pricing.SUPABASE_PRO_BASE + storage_cost + bandwidth_cost                # Vercel costs        vercel_base = self.pricing.VERCEL_PRO_PER_SEAT * vercel_team_members if use_vercel else 0        vercel_bandwidth_overage = max(0, frontend_bandwidth_gb - self.pricing.VERCEL_BANDWIDTH_INCLUDED) if use_vercel else 0        vercel_bandwidth_cost = vercel_bandwidth_overage * self.pricing.VERCEL_BANDWIDTH_COST        vercel_total = vercel_base + vercel_bandwidth_cost                return {            'total_images': total_images,            'total_storage_gb': total_storage_gb,            'image_bandwidth_gb': image_bandwidth_gb,            'frontend_bandwidth_gb': frontend_bandwidth_gb,            'supabase_base': self.pricing.SUPABASE_PRO_BASE,            'storage_cost': storage_cost,            'bandwidth_cost': bandwidth_cost,            'supabase_total': supabase_total,            'vercel_base': vercel_base,            'vercel_bandwidth_cost': vercel_bandwidth_cost,            'vercel_total': vercel_total,            'infrastructure_total': supabase_total + vercel_total        }        def calculate_unit_economics(self, customers, price_per_customer, total_costs):        """Calculate per-customer/per-client economics"""        if customers == 0:            return {                'revenue_per_customer': 0,                'cost_per_customer': 0,                'profit_per_customer': 0,                'unit_economics_ratio': 0            }                revenue_per_customer = price_per_customer        cost_per_customer = total_costs / customers        profit_per_customer = revenue_per_customer - cost_per_customer        unit_economics_ratio = revenue_per_customer / cost_per_customer if cost_per_customer > 0 else np.inf                return {            'revenue_per_customer': revenue_per_customer,            'cost_per_customer': cost_per_customer,            'profit_per_customer': profit_per_customer,            'unit_economics_ratio': unit_economics_ratio        }        def calculate_ltv_cac(self, price_per_customer, monthly_churn_pct,                          marketing_spend, customers, monthly_growth_pct):        """        Calculate Customer/Client Lifetime Value (LTV) and Customer Acquisition Cost (CAC).                Edge case handling:        - Zero churn: Caps lifetime at MAX_LIFETIME_MONTHS instead of infinity        - Zero growth: Uses MIN_GROWTH_FOR_CAC to avoid division by zero in CAC        - Zero marketing with zero growth: Returns CAC of 0 (no acquisition cost)        - Invalid ratios: Returns np.nan instead of arbitrary numbers                Returns dict with ltv, cac, ltv_cac_ratio, and avg_lifetime_months        """                # LTV calculation        if monthly_churn_pct > 0:            avg_lifetime_months = 1 / (monthly_churn_pct / 100)            # Cap at reasonable maximum to avoid infinity in calculations            avg_lifetime_months = min(avg_lifetime_months, self.pricing.MAX_LIFETIME_MONTHS)        else:            # Zero churn = customers stay forever, but cap at 10 years            avg_lifetime_months = self.pricing.MAX_LIFETIME_MONTHS                ltv = price_per_customer * avg_lifetime_months                # CAC calculation        # Handle zero growth: use minimum growth rate to avoid division by zero        effective_growth = max(monthly_growth_pct, self.pricing.MIN_GROWTH_FOR_CAC)        new_customers_per_month = customers * (effective_growth / 100)                if new_customers_per_month > 0:            cac = marketing_spend / new_customers_per_month        else:            # No new customers: CAC is zero if no spend, infinite if spending            cac = 0 if marketing_spend == 0 else np.inf                # LTV:CAC Ratio calculation        if cac > 0 and np.isfinite(cac):            ltv_cac_ratio = ltv / cac        elif cac == 0 and ltv > 0:            # Perfect case: LTV but no acquisition cost            ltv_cac_ratio = np.inf        else:            # Invalid case: can't calculate meaningful ratio            ltv_cac_ratio = np.nan                return {            'ltv': ltv,            'cac': cac,            'ltv_cac_ratio': ltv_cac_ratio,            'avg_lifetime_months': avg_lifetime_months        }calculator = BusinessCalculator(pricing)print("✓ Business calculator initialized with improved edge case handling")

In [ ]:
# Interactive calculator with configuration objectfrom dataclasses import dataclass@dataclassclass CalculatorConfig:    """Configuration for film location database platform calculator"""    # Client & Usage    customers: int = 50    locations_per_customer: int = 150    images_per_location: int = 25    avg_image_size_mb: float = 4.0    monthly_views_per_image: int = 5        # Pricing & Revenue    price_per_customer: int = 499    monthly_churn_pct: float = 3.0    monthly_growth_pct: float = 8.0        # Infrastructure    use_vercel: bool = True    vercel_team_members: int = 1    vercel_bandwidth_mult: float = 0.3        # Operating Costs    employee_salaries: int = 15000    marketing_spend: int = 3000    other_costs: int = 500def interactive_calculator(config: CalculatorConfig = None):    """    Film Industry Location Database Platform Calculator        Args:        config: CalculatorConfig object with all parameters                Defaults optimized for production company/professional tier        Configuration defaults:    - 50 active clients (production companies, agencies)    - 150 locations per client (typical professional database)    - 25 high-res images per location (multiple angles, interiors/exteriors)    - 4MB average image size (film industry quality standards)    - 5 monthly views per image (focused scouting/presentation usage)    - $499/month pricing (Professional tier)    - 3% monthly churn (annual contracts typical in B2B)    - 8% monthly growth (enterprise sales cycle)    - $15K team costs (developers, customer success, location curators)    - $3K marketing (film festivals, trade shows, industry partnerships)    - $500 other costs (legal, insurance, tools)    """        # Use default config if none provided    if config is None:        config = CalculatorConfig()        # Calculate infrastructure costs    infra = calculator.calculate_infrastructure_costs(        config.customers, config.locations_per_customer, config.images_per_location,        config.avg_image_size_mb, config.monthly_views_per_image, config.use_vercel,        config.vercel_team_members, config.vercel_bandwidth_mult    )        # Total costs    total_costs = infra['infrastructure_total'] + config.employee_salaries + config.marketing_spend + config.other_costs        # Revenue    monthly_revenue = config.customers * config.price_per_customer    net_profit = monthly_revenue - total_costs    profit_margin = (net_profit / monthly_revenue * 100) if monthly_revenue > 0 else 0    gross_margin = ((monthly_revenue - infra['infrastructure_total']) / monthly_revenue * 100) if monthly_revenue > 0 else 0        # Unit economics    unit_econ = calculator.calculate_unit_economics(config.customers, config.price_per_customer, total_costs)        # LTV/CAC    ltv_cac = calculator.calculate_ltv_cac(config.price_per_customer, config.monthly_churn_pct,                                            config.marketing_spend, config.customers, config.monthly_growth_pct)        # Display results    print("="*80)    print(" " * 20 + "FILM LOCATION PLATFORM - BUSINESS RESULTS")    print("="*80)        print("\nREVENUE & PROFITABILITY")    print("-" * 80)    print(f"Monthly Recurring Revenue (MRR):        ${monthly_revenue:,.0f}")    print(f"Annual Recurring Revenue (ARR):         ${monthly_revenue * 12:,.0f}")    print(f"Net Profit:                             ${net_profit:,.0f}")    print(f"Profit Margin:                          {profit_margin:.1f}%")    profit_health = "Excellent" if profit_margin >= pricing.SAAS_EXCELLENT_MARGIN else "Good" if profit_margin >= pricing.SAAS_GOOD_MARGIN else "Needs Improvement"    print(f"Margin Health:                          {profit_health}")        print("\nINFRASTRUCTURE & USAGE")    print("-" * 80)    print(f"Total Images:                           {infra['total_images']:,.0f}")    print(f"Total Storage:                          {infra['total_storage_gb']:.2f} GB")    print(f"Image Bandwidth (Supabase):             {infra['image_bandwidth_gb']:.2f} GB")    print(f"Frontend Bandwidth (Vercel):            {infra['frontend_bandwidth_gb']:.2f} GB")        print("\nCOST BREAKDOWN")    print("-" * 80)    print(f"Supabase Base:                          ${infra['supabase_base']:.2f}")    print(f"Supabase Storage Overage:               ${infra['storage_cost']:.2f}")    print(f"Supabase Bandwidth Overage:             ${infra['bandwidth_cost']:.2f}")    print(f"Supabase Total:                         ${infra['supabase_total']:.2f}")    print()    print(f"Vercel Base:                            ${infra['vercel_base']:.2f}")    print(f"Vercel Bandwidth Overage:               ${infra['vercel_bandwidth_cost']:.2f}")    print(f"Vercel Total:                           ${infra['vercel_total']:.2f}")    print()    print(f"Team Costs:                             ${config.employee_salaries:.2f}")    print(f"Marketing & Sales:                      ${config.marketing_spend:.2f}")    print(f"Other Operating Costs:                  ${config.other_costs:.2f}")    print(f"\n{'TOTAL MONTHLY COSTS:'.ljust(40)} ${total_costs:,.2f}")        print("\nUNIT ECONOMICS")    print("-" * 80)    print(f"Revenue per Client:                     ${unit_econ['revenue_per_customer']:.2f}")    print(f"Cost per Client:                        ${unit_econ['cost_per_customer']:.2f}")    print(f"Profit per Client:                      ${unit_econ['profit_per_customer']:.2f}")        # Handle infinite ratio display    if np.isfinite(unit_econ['unit_economics_ratio']):        print(f"Unit Economics Ratio:                   {unit_econ['unit_economics_ratio']:.2f}x")    else:        print(f"Unit Economics Ratio:                   ∞ (infinite - no cost per customer)")        print(f"Gross Margin:                           {gross_margin:.1f}%")        print("\nCLIENT METRICS")    print("-" * 80)    print(f"Client Lifetime Value (LTV):            ${ltv_cac['ltv']:.0f}")        # Handle infinite/NaN CAC display    if np.isfinite(ltv_cac['cac']):        print(f"Client Acquisition Cost (CAC):          ${ltv_cac['cac']:.0f}")    elif ltv_cac['cac'] == 0:        print(f"Client Acquisition Cost (CAC):          $0 (no marketing spend)")    else:        print(f"Client Acquisition Cost (CAC):          ∞ (spending without growth)")        # Handle LTV:CAC ratio display    if np.isfinite(ltv_cac['ltv_cac_ratio']):        print(f"LTV:CAC Ratio:                          {ltv_cac['ltv_cac_ratio']:.1f}:1")        ltv_cac_health = "Excellent (>3:1)" if ltv_cac['ltv_cac_ratio'] >= pricing.SAAS_GOOD_LTV_CAC_RATIO else "Acceptable (>1:1)" if ltv_cac['ltv_cac_ratio'] > pricing.SAAS_MIN_LTV_CAC_RATIO else "Needs Improvement (<1:1)"        print(f"LTV:CAC Health:                         {ltv_cac_health}")    elif np.isnan(ltv_cac['ltv_cac_ratio']):        print(f"LTV:CAC Ratio:                          N/A (invalid inputs)")    else:        print(f"LTV:CAC Ratio:                          ∞:1 (perfect - organic growth)")        print(f"Avg Client Lifetime:                    {ltv_cac['avg_lifetime_months']:.1f} months")    print(f"Monthly Churn:                          {config.monthly_churn_pct}%")        # Break-even    break_even_customers = np.ceil(total_costs / config.price_per_customer) if config.price_per_customer > 0 else 0    print("\nBREAK-EVEN ANALYSIS")    print("-" * 80)    print(f"Clients Needed to Break Even:           {break_even_customers:.0f}")    print(f"MRR Needed to Break Even:               ${total_costs:,.0f}")    print(f"Current Status:                         {'Profitable' if config.customers >= break_even_customers else 'Below Break-Even'}")        print("\n" + "="*80)        return {        'infra': infra,        'revenue': monthly_revenue,        'costs': total_costs,        'profit': net_profit,        'unit_econ': unit_econ,        'ltv_cac': ltv_cac    }# Create interactive widget with individual parameters (for backward compatibility)def _widget_wrapper(customers, locations_per_customer, images_per_location,                   avg_image_size_mb, monthly_views_per_image, price_per_customer,                   monthly_churn_pct, monthly_growth_pct, use_vercel, vercel_team_members,                   vercel_bandwidth_mult, employee_salaries, marketing_spend, other_costs):    """Wrapper to convert widget parameters to config object"""    config = CalculatorConfig(        customers=customers,        locations_per_customer=locations_per_customer,        images_per_location=images_per_location,        avg_image_size_mb=avg_image_size_mb,        monthly_views_per_image=monthly_views_per_image,        price_per_customer=price_per_customer,        monthly_churn_pct=monthly_churn_pct,        monthly_growth_pct=monthly_growth_pct,        use_vercel=use_vercel,        vercel_team_members=vercel_team_members,        vercel_bandwidth_mult=vercel_bandwidth_mult,        employee_salaries=employee_salaries,        marketing_spend=marketing_spend,        other_costs=other_costs    )    return interactive_calculator(config)# Create interactive widget with film industry rangesinteractive_calc = interactive(    _widget_wrapper,    customers=widgets.IntSlider(min=1, max=500, step=5, value=50, description='Clients:'),    locations_per_customer=widgets.IntSlider(min=10, max=500, step=10, value=150, description='Locations/Client:'),    images_per_location=widgets.IntSlider(min=5, max=100, step=5, value=25, description='Images/Location:'),    avg_image_size_mb=widgets.FloatSlider(min=1.0, max=10, step=0.5, value=4.0, description='Image Size (MB):'),    monthly_views_per_image=widgets.IntSlider(min=1, max=50, step=1, value=5, description='Views/Image:'),    price_per_customer=widgets.IntSlider(min=99, max=5000, step=50, value=499, description='Price/Client:'),    monthly_churn_pct=widgets.FloatSlider(min=0, max=15, step=0.5, value=3.0, description='Churn %:'),    monthly_growth_pct=widgets.FloatSlider(min=0, max=30, step=1.0, value=8.0, description='Growth %:'),    use_vercel=widgets.Checkbox(value=True, description='Use Vercel'),    vercel_team_members=widgets.IntSlider(min=1, max=10, step=1, value=1, description='Vercel Seats:'),    vercel_bandwidth_mult=widgets.FloatSlider(min=0, max=1, step=0.05, value=0.3, description='Frontend BW %:'),    employee_salaries=widgets.IntSlider(min=0, max=100000, step=1000, value=15000, description='Team Costs:'),    marketing_spend=widgets.IntSlider(min=0, max=20000, step=500, value=3000, description='Marketing:'),    other_costs=widgets.IntSlider(min=0, max=5000, step=100, value=500, description='Operating Costs:'))display(interactive_calc)

## 4. Revenue & Cost Projections <a id="projections"></a>

12-month forward-looking projections with growth and churn modeling.

In [ ]:
def project_12_months(starting_customers, locations_per_customer, images_per_location,
                     avg_image_size_mb, monthly_views_per_image, price_per_customer,
                     monthly_churn_pct, monthly_growth_pct, use_vercel, vercel_team_members,
                     vercel_bandwidth_mult, employee_salaries, marketing_spend, other_costs):
    """Project business metrics for 12 months"""
    
    months = []
    customers_list = []
    revenue_list = []
    costs_list = []
    profit_list = []
    storage_list = []
    bandwidth_list = []
    
    current_customers = starting_customers
    
    for month in range(1, 13):
        # Customer dynamics
        new_customers = current_customers * (monthly_growth_pct / 100)
        churned_customers = current_customers * (monthly_churn_pct / 100)
        current_customers = current_customers + new_customers - churned_customers
        
        # Calculate costs for this month
        infra = calculator.calculate_infrastructure_costs(
            current_customers, locations_per_customer, images_per_location,
            avg_image_size_mb, monthly_views_per_image, use_vercel,
            vercel_team_members, vercel_bandwidth_mult
        )
        
        total_costs = infra['infrastructure_total'] + employee_salaries + marketing_spend + other_costs
        revenue = current_customers * price_per_customer
        profit = revenue - total_costs
        
        # Store results
        months.append(month)
        customers_list.append(current_customers)
        revenue_list.append(revenue)
        costs_list.append(total_costs)
        profit_list.append(profit)
        storage_list.append(infra['total_storage_gb'])
        bandwidth_list.append(infra['image_bandwidth_gb'] + infra['frontend_bandwidth_gb'])
    
    return pd.DataFrame({
        'Month': months,
        'Customers': customers_list,
        'Revenue': revenue_list,
        'Costs': costs_list,
        'Profit': profit_list,
        'Storage_GB': storage_list,
        'Bandwidth_GB': bandwidth_list
    })

# Generate projection with default values
projection_df = project_12_months(
    starting_customers=100,
    locations_per_customer=10,
    images_per_location=20,
    avg_image_size_mb=2.0,
    monthly_views_per_image=10,
    price_per_customer=49,
    monthly_churn_pct=5.0,
    monthly_growth_pct=10.0,
    use_vercel=True,
    vercel_team_members=1,
    vercel_bandwidth_mult=0.3,
    employee_salaries=0,
    marketing_spend=0,
    other_costs=0
)

# Display summary
print("\n📅 12-MONTH PROJECTION SUMMARY")
print("="*80)
print(f"Starting Customers:      {projection_df['Customers'].iloc[0]:.0f}")
print(f"Ending Customers:        {projection_df['Customers'].iloc[-1]:.0f}")
print(f"Customer Growth:         {((projection_df['Customers'].iloc[-1] / projection_df['Customers'].iloc[0] - 1) * 100):.1f}%")
print(f"\nTotal Revenue (12mo):    ${projection_df['Revenue'].sum():,.0f}")
print(f"Total Costs (12mo):      ${projection_df['Costs'].sum():,.0f}")
print(f"Total Profit (12mo):     ${projection_df['Profit'].sum():,.0f}")
print(f"\nAverage Monthly MRR:     ${projection_df['Revenue'].mean():,.0f}")
print(f"Ending MRR (Month 12):   ${projection_df['Revenue'].iloc[-1]:,.0f}")
print("="*80)

# Display table
display_df = projection_df.copy()
display_df['Customers'] = display_df['Customers'].round(0).astype(int)
display_df['Revenue'] = display_df['Revenue'].round(0).apply(lambda x: f"${x:,.0f}")
display_df['Costs'] = display_df['Costs'].round(0).apply(lambda x: f"${x:,.0f}")
display_df['Profit'] = display_df['Profit'].round(0).apply(lambda x: f"${x:,.0f}")
display_df['Storage_GB'] = display_df['Storage_GB'].round(1)
display_df['Bandwidth_GB'] = display_df['Bandwidth_GB'].round(1)

display(HTML(display_df.to_html(index=False)))

In [ ]:
# Visualize projections
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Revenue vs Costs Over Time', 'Customer Growth',
                    'Profit Margin Trend', 'Infrastructure Usage'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": True}, {"secondary_y": True}]]
)

# Plot 1: Revenue vs Costs
fig.add_trace(
    go.Scatter(x=projection_df['Month'], y=projection_df['Revenue'],
               name='Revenue', line=dict(color=COLORS['success'], width=3)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=projection_df['Month'], y=projection_df['Costs'],
               name='Costs', line=dict(color=COLORS['danger'], width=3)),
    row=1, col=1
)

# Plot 2: Customer Growth
fig.add_trace(
    go.Scatter(x=projection_df['Month'], y=projection_df['Customers'],
               name='Customers', fill='tozeroy',
               line=dict(color=COLORS['primary'], width=3)),
    row=1, col=2
)

# Plot 3: Profit Margin
profit_margin = (projection_df['Profit'] / projection_df['Revenue'] * 100)
fig.add_trace(
    go.Scatter(x=projection_df['Month'], y=profit_margin,
               name='Profit Margin %', line=dict(color=COLORS['info'], width=3)),
    row=2, col=1
)
fig.add_hline(y=20, line_dash="dash", line_color="green",
              annotation_text="Target: 20%", row=2, col=1)

# Plot 4: Infrastructure Usage
fig.add_trace(
    go.Scatter(x=projection_df['Month'], y=projection_df['Storage_GB'],
               name='Storage (GB)', line=dict(color=COLORS['warning'], width=2)),
    row=2, col=2
)
fig.add_trace(
    go.Scatter(x=projection_df['Month'], y=projection_df['Bandwidth_GB'],
               name='Bandwidth (GB)', line=dict(color=COLORS['secondary'], width=2)),
    row=2, col=2, secondary_y=True
)

# Update layout
fig.update_layout(
    height=800,
    showlegend=True,
    title_text="12-Month Business Projections",
    title_font_size=20
)

fig.update_xaxes(title_text="Month", row=1, col=1)
fig.update_xaxes(title_text="Month", row=1, col=2)
fig.update_xaxes(title_text="Month", row=2, col=1)
fig.update_xaxes(title_text="Month", row=2, col=2)

fig.update_yaxes(title_text="Amount ($)", row=1, col=1)
fig.update_yaxes(title_text="Customers", row=1, col=2)
fig.update_yaxes(title_text="Margin (%)", row=2, col=1)
fig.update_yaxes(title_text="Storage (GB)", row=2, col=2)
fig.update_yaxes(title_text="Bandwidth (GB)", row=2, col=2, secondary_y=True)

fig.show()

## 5. Scenario Analysis <a id="scenarios"></a>

Compare different business scenarios: Conservative, Base Case, and Aggressive Growth.

In [ ]:
# Define scenarios
scenarios = {
    'Conservative': {
        'monthly_growth_pct': 5,
        'monthly_churn_pct': 7,
        'price_per_customer': 39
    },
    'Base Case': {
        'monthly_growth_pct': 10,
        'monthly_churn_pct': 5,
        'price_per_customer': 49
    },
    'Aggressive': {
        'monthly_growth_pct': 20,
        'monthly_churn_pct': 3,
        'price_per_customer': 69
    }
}

# Base parameters
base_params = {
    'starting_customers': 100,
    'locations_per_customer': 10,
    'images_per_location': 20,
    'avg_image_size_mb': 2.0,
    'monthly_views_per_image': 10,
    'use_vercel': True,
    'vercel_team_members': 1,
    'vercel_bandwidth_mult': 0.3,
    'employee_salaries': 0,
    'marketing_spend': 1000,
    'other_costs': 100
}

# Generate projections for each scenario
scenario_results = {}
for scenario_name, scenario_params in scenarios.items():
    params = {**base_params, **scenario_params}
    df = project_12_months(**params)
    scenario_results[scenario_name] = df

# Compare scenarios at month 12
comparison_data = []
for scenario_name, df in scenario_results.items():
    final_month = df.iloc[-1]
    comparison_data.append({
        'Scenario': scenario_name,
        'Customers': int(final_month['Customers']),
        'Monthly Revenue': f"${final_month['Revenue']:,.0f}",
        'Monthly Costs': f"${final_month['Costs']:,.0f}",
        'Monthly Profit': f"${final_month['Profit']:,.0f}",
        'Profit Margin': f"{(final_month['Profit'] / final_month['Revenue'] * 100):.1f}%",
        'Total 12mo Revenue': f"${df['Revenue'].sum():,.0f}",
        'Total 12mo Profit': f"${df['Profit'].sum():,.0f}"
    })

comparison_df = pd.DataFrame(comparison_data)

print("\n🎯 SCENARIO COMPARISON (Month 12)")
print("="*80)
display(HTML(comparison_df.to_html(index=False)))

In [ ]:
# Visualize scenario comparison
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Revenue Trajectory by Scenario', 'Customer Growth by Scenario')
)

colors_list = [COLORS['success'], COLORS['primary'], COLORS['warning']]

for idx, (scenario_name, df) in enumerate(scenario_results.items()):
    fig.add_trace(
        go.Scatter(x=df['Month'], y=df['Revenue'],
                   name=scenario_name, line=dict(color=colors_list[idx], width=3)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=df['Month'], y=df['Customers'],
                   name=scenario_name, line=dict(color=colors_list[idx], width=3),
                   showlegend=False),
        row=1, col=2
    )

fig.update_layout(
    height=500,
    title_text="Scenario Analysis: 12-Month Comparison",
    title_font_size=20
)

fig.update_xaxes(title_text="Month")
fig.update_yaxes(title_text="Revenue ($)", row=1, col=1)
fig.update_yaxes(title_text="Customers", row=1, col=2)

fig.show()

## 6. Unit Economics & Metrics <a id="metrics"></a>

Deep dive into per-customer economics and SaaS metrics.

In [ ]:
# Pricing sensitivity analysis - Film industry tiers# Use film industry pricing ranges and typical usage patternsprice_points = [149, 299, 499, 799, 1499, 2999, 3999]  # Film industry tier pricescustomers = 50  # Typical starting client basepricing_analysis = []for price in price_points:    # Use film industry defaults: 150 locations, 25 images, 4MB, 5 views    infra = calculator.calculate_infrastructure_costs(        customers, 150, 25, 4.0, 5, True, 1, 0.3    )    total_costs = infra['infrastructure_total'] + pricing.OTHER_FIXED_COSTS    revenue = customers * price    profit = revenue - total_costs    margin = (profit / revenue * 100) if revenue > 0 else 0    break_even = np.ceil(total_costs / price)        pricing_analysis.append({        'Price': price,        'Revenue': revenue,        'Profit': profit,        'Margin': margin,        'Break_Even_Customers': break_even    })pricing_df = pd.DataFrame(pricing_analysis)# Visualize pricing analysisfig = make_subplots(    rows=1, cols=2,    subplot_titles=('Profit Margin by Price Point', 'Break-Even Clients by Price'))fig.add_trace(    go.Bar(x=pricing_df['Price'], y=pricing_df['Margin'],           marker_color=COLORS['success'], name='Margin %'),    row=1, col=1)fig.add_trace(    go.Scatter(x=pricing_df['Price'], y=pricing_df['Break_Even_Customers'],               mode='lines+markers', line=dict(color=COLORS['danger'], width=3),               name='Break-Even Clients'),    row=1, col=2)fig.update_layout(    height=500,    title_text="Film Industry Pricing Strategy Analysis",    title_font_size=20,    showlegend=False)fig.update_xaxes(title_text="Price per Client ($)")fig.update_yaxes(title_text="Profit Margin (%)", row=1, col=1)fig.update_yaxes(title_text="Clients Needed", row=1, col=2)fig.show()# Display pricing tabledisplay_pricing = pricing_df.copy()display_pricing['Price'] = display_pricing['Price'].apply(lambda x: f"${x}")display_pricing['Revenue'] = display_pricing['Revenue'].apply(lambda x: f"${x:,.0f}")display_pricing['Profit'] = display_pricing['Profit'].apply(lambda x: f"${x:,.0f}")display_pricing['Margin'] = display_pricing['Margin'].apply(lambda x: f"{x:.1f}%")display_pricing['Break_Even_Customers'] = display_pricing['Break_Even_Customers'].astype(int)print("\n💰 PRICING SENSITIVITY ANALYSIS")print(f"Operating costs: ${pricing.OTHER_FIXED_COSTS:,}/month (team + marketing + other)")display(HTML(display_pricing.to_html(index=False)))

In [ ]:
# Cohort LTV analysis
churn_rates = [2, 3, 5, 7, 10]
prices = [29, 49, 69, 99]

ltv_matrix = []
for churn in churn_rates:
    row = []
    avg_lifetime = 1 / (churn / 100)
    for price in prices:
        ltv = price * avg_lifetime
        row.append(ltv)
    ltv_matrix.append(row)

ltv_df = pd.DataFrame(ltv_matrix, columns=[f'${p}/mo' for p in prices],
                      index=[f'{c}% churn' for c in churn_rates])

print("\n📊 CUSTOMER LIFETIME VALUE (LTV) MATRIX")
print("="*80)
print("Shows LTV for different price and churn combinations\n")

# Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=ltv_df.values,
    x=ltv_df.columns,
    y=ltv_df.index,
    colorscale='Viridis',
    text=ltv_df.values.round(0),
    texttemplate='$%{text}',
    textfont={"size":12},
    colorbar=dict(title="LTV ($)")
))

fig.update_layout(
    title='Customer Lifetime Value Heatmap',
    xaxis_title='Monthly Price',
    yaxis_title='Monthly Churn Rate',
    height=500
)

fig.show()

# Display table
display_ltv = ltv_df.copy()
display_ltv = display_ltv.applymap(lambda x: f"${x:,.0f}")
display(HTML(display_ltv.to_html()))

## 7. Break-Even Analysis <a id="breakeven"></a>

Determine when your business becomes profitable.

In [ ]:
# Break-even chart - Film industry professional tiercustomer_range = range(1, 201, 10)price_per_customer = pricing.TIER_PROFESSIONAL  # $499 professional tierfixed_costs = pricing.SUPABASE_PRO_BASE + pricing.VERCEL_PRO_PER_SEAT + pricing.OTHER_FIXED_COSTSrevenue_curve = []cost_curve = []for customers in customer_range:    # Film industry defaults: 150 locations, 25 images, 4MB, 5 views    infra = calculator.calculate_infrastructure_costs(        customers, 150, 25, 4.0, 5, True, 1, 0.3    )    total_costs = infra['infrastructure_total'] + pricing.OTHER_FIXED_COSTS    revenue = customers * price_per_customer        revenue_curve.append(revenue)    cost_curve.append(total_costs)# Find break-even pointbreak_even_idx = Nonefor i, (rev, cost) in enumerate(zip(revenue_curve, cost_curve)):    if rev >= cost:        break_even_idx = i        breakbreak_even_customers = list(customer_range)[break_even_idx] if break_even_idx else None# Create visualizationfig = go.Figure()fig.add_trace(go.Scatter(    x=list(customer_range),    y=revenue_curve,    name='Revenue',    line=dict(color=COLORS['success'], width=3)))fig.add_trace(go.Scatter(    x=list(customer_range),    y=cost_curve,    name='Total Costs',    line=dict(color=COLORS['danger'], width=3)))if break_even_customers:    fig.add_vline(        x=break_even_customers,        line_dash="dash",        line_color="gray",        annotation_text=f"Break-Even: {break_even_customers} clients",        annotation_position="top right"    )fig.update_layout(    title='Break-Even Analysis - Film Industry Professional Tier',    xaxis_title='Number of Clients',    yaxis_title='Monthly Amount ($)',    height=600,    hovermode='x unified')fig.show()print(f"\n⚖️  BREAK-EVEN ANALYSIS (Professional Tier)")print("="*80)print(f"Price per Client:                 ${price_per_customer}")print(f"Fixed Monthly Costs:              ${fixed_costs:,.0f}")print(f"  - Infrastructure base:          ${pricing.SUPABASE_PRO_BASE + pricing.VERCEL_PRO_PER_SEAT}")print(f"  - Operating costs:              ${pricing.OTHER_FIXED_COSTS:,}")print(f"Break-Even Clients:               {break_even_customers}")print(f"Break-Even MRR:                   ${break_even_customers * price_per_customer:,.0f}")print("="*80)

## 8. Strategic Recommendations <a id="recommendations"></a>

Actionable insights based on your business model.

In [ ]:
def generate_recommendations(customers, price, costs, revenue, churn_pct, growth_pct):
    """Generate strategic recommendations based on business metrics"""
    
    recommendations = []
    
    profit = revenue - costs
    margin = (profit / revenue * 100) if revenue > 0 else 0
    
    print("\n🎯 STRATEGIC RECOMMENDATIONS")
    print("="*80)
    
    # Profitability recommendations
    if margin < 0:
        print("\n🔴 CRITICAL: Unprofitable Business")
        print("-" * 80)
        target_price = np.ceil((costs / customers) * 1.3)
        print(f"• Increase pricing to ${target_price:.0f}/month (+{((target_price/price - 1)*100):.0f}%)")
        print(f"• Acquire {np.ceil(costs/price - customers):.0f} more customers at current pricing")
        print(f"• Reduce costs by ${abs(profit):.0f}/month")
    elif margin < 20:
        print("\n🟡 LOW MARGIN: Below SaaS Benchmarks")
        print("-" * 80)
        print(f"• Current margin: {margin:.1f}% (Target: 20%+)")
        print(f"• Consider price increase of ${(costs / customers * 1.25 - price):.0f}/month")
        print("• Implement cost optimization strategies")
    else:
        print("\n🟢 HEALTHY MARGINS")
        print("-" * 80)
        print(f"• Current margin: {margin:.1f}% - Excellent!")
        print("• Focus on sustainable growth")
    
    # Churn recommendations
    if churn_pct > 7:
        print("\n🔴 HIGH CHURN RATE")
        print("-" * 80)
        print(f"• Current churn: {churn_pct}% (Target: <5%)")
        print("• Improve onboarding and customer success")
        print("• Conduct customer interviews to identify pain points")
        print("• Add features that increase product stickiness")
        ltv_improvement = (1/(churn_pct/100) - 1/(5/100)) * price
        print(f"• Reducing churn to 5% would increase LTV by ${ltv_improvement:.0f}")
    elif churn_pct > 5:
        print("\n🟡 MODERATE CHURN")
        print("-" * 80)
        print(f"• Current churn: {churn_pct}% (Target: <5%)")
        print("• Monitor churn metrics closely")
        print("• Implement customer retention initiatives")
    else:
        print("\n🟢 EXCELLENT RETENTION")
        print("-" * 80)
        print(f"• Current churn: {churn_pct}% - Best in class!")
        print("• Maintain focus on customer success")
    
    # Growth recommendations
    if growth_pct < 5:
        print("\n🟡 SLOW GROWTH")
        print("-" * 80)
        print(f"• Current growth: {growth_pct}% (Consider: 10%+)")
        print("• Increase marketing spend to accelerate acquisition")
        print("• Implement referral program")
        print("• Improve conversion funnel")
    elif growth_pct > 15:
        print("\n🟢 STRONG GROWTH")
        print("-" * 80)
        print(f"• Current growth: {growth_pct}% - Excellent!")
        print("• Ensure infrastructure can scale")
        print("• Monitor unit economics as you scale")
        print("• Consider increasing team size")
    
    # Cost optimization
    print("\n💡 COST OPTIMIZATION OPPORTUNITIES")
    print("-" * 80)
    print("• Implement image compression (WebP format, ~30% storage savings)")
    print("• Use CDN caching aggressively (40%+ bandwidth reduction)")
    print("• Implement lazy loading for images")
    print("• Consider tiered storage (hot/cold) for older images")
    print("• Optimize image delivery with responsive sizes")
    
    # Pricing strategy
    print("\n💰 PRICING STRATEGY")
    print("-" * 80)
    if price < 40:
        print("• Consider implementing tiered pricing:")
        print(f"  - Starter: $29/mo (limited features)")
        print(f"  - Professional: ${price + 20}/mo (current features)")
        print(f"  - Enterprise: ${price + 50}/mo (premium features)")
    else:
        print("• Premium pricing - ensure value proposition is clear")
        print("• Consider annual plans with discount (10-15% off)")
        print("• Add enterprise tier for high-volume customers")
    
    print("\n" + "="*80)

# Generate recommendations for base case
generate_recommendations(
    customers=100,
    price=49,
    costs=1380,
    revenue=4900,
    churn_pct=5.0,
    growth_pct=10.0
)

## 9. Export Business Plan <a id="export"></a>

Generate a comprehensive business plan document.

In [ ]:
def generate_business_plan_summary():
    """Generate a comprehensive business plan summary"""
    
    plan = f"""
# SaaS Photo Platform Business Plan
## Financial Model & Projections

---

## Executive Summary

This document outlines the financial model and business plan for a SaaS photo storage and
serving platform targeting businesses that need to manage location-based photo collections.

### Technology Stack
- **Backend**: Supabase (PostgreSQL, Storage, Auth)
- **Frontend Hosting**: Vercel (Next.js/React)
- **Infrastructure**: Cloud-native, serverless architecture

### Target Market
- Real estate agencies
- Property management companies  
- Retail chains with multiple locations
- Construction and inspection services

---

## Financial Model

### Cost Structure

**Fixed Costs:**
- Supabase Pro: $120/month (includes 100GB storage, 250GB bandwidth)
- Vercel Pro: $20/month per seat
- Team salaries and contractors
- Marketing and operations

**Variable Costs:**
- Storage: $0.021/GB over 100GB
- Bandwidth (Supabase): $0.09/GB over 250GB
- Bandwidth (Vercel): $0.15/GB over 1TB

### Revenue Model

**Pricing Tiers:**
- Starter: $29/month (up to 5 locations, 100 images)
- Professional: $49/month (up to 20 locations, 500 images)
- Business: $79/month (up to 50 locations, 2000 images)
- Enterprise: Custom pricing (unlimited)

### Unit Economics (Professional Tier)

- Average Revenue per Customer: $49/month
- Average Cost per Customer: ~$14-18/month
- Gross Margin: ~70%
- Target LTV: $980 (assuming 5% monthly churn)
- Target CAC: $150-200 (LTV:CAC = 5:1)

---

## 12-Month Projections

**Base Case Assumptions:**
- Starting customers: 100
- Monthly growth: 10%
- Monthly churn: 5%
- Average price: $49/month

**Expected Outcomes (Month 12):**
- Customers: ~170
- MRR: ~$8,330
- ARR: ~$100,000
- Monthly profit: $4,000-5,000
- Profit margin: 40-50%

---

## Key Metrics & Benchmarks

**Target SaaS Metrics:**
- Monthly Churn: <5% (Industry: 5-7%)
- Gross Margin: >70% (Industry: 70-85%)
- LTV:CAC Ratio: >3:1 (Healthy: 3:1 to 5:1)
- CAC Payback: <12 months (Industry: 12-18 months)
- Net Revenue Retention: >100%

**Growth Targets:**
- Year 1: 500 customers, $300K ARR
- Year 2: 2,000 customers, $1.2M ARR
- Year 3: 5,000 customers, $3M ARR

---

## Go-to-Market Strategy

### Customer Acquisition
1. **Content Marketing**: SEO-optimized content targeting property managers
2. **Paid Advertising**: Google Ads, LinkedIn targeting decision-makers
3. **Partnerships**: Integrate with property management software
4. **Referral Program**: Incentivize existing customers

### Retention Strategy
1. **Onboarding**: White-glove setup for first 50 customers
2. **Customer Success**: Proactive monitoring and support
3. **Product Development**: Regular feature releases based on feedback
4. **Community**: User forum and best practices sharing

---

## Risk Analysis

### Key Risks
1. **Technical**: Infrastructure costs scaling faster than revenue
2. **Market**: Competition from established players
3. **Operational**: High customer acquisition costs
4. **Product**: Slow product-market fit

### Mitigation Strategies
1. **Cost Optimization**: Aggressive caching, compression, tiered storage
2. **Differentiation**: Focus on vertical-specific features
3. **Efficient CAC**: Content marketing, partnerships, referrals
4. **Rapid Iteration**: Monthly releases, customer feedback loops

---

## Funding Requirements

### Bootstrap Scenario (Recommended)
- Initial investment: $50K-75K
- Runway: 12-18 months to profitability
- Focus on capital efficiency and early revenue

### Venture-Backed Scenario
- Seed round: $500K-1M
- Use of funds: Team (60%), Marketing (30%), Operations (10%)
- Milestone: $1M ARR in 18-24 months

---

## Conclusion

The SaaS photo platform presents a compelling business opportunity with:
- **Strong unit economics** (70%+ gross margins)
- **Scalable infrastructure** (cloud-native, variable costs)
- **Clear target market** (property management, real estate)
- **Path to profitability** (within 12-18 months)

Key success factors include maintaining low churn, efficient customer acquisition,
and continuous product improvement based on customer feedback.

---

*Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}*
"""
    
    return plan

# Generate and display business plan
business_plan = generate_business_plan_summary()
display(Markdown(business_plan))

In [ ]:
# Export business plan to markdown file
with open('Business_Plan_SaaS_Photo_Platform.md', 'w') as f:
    f.write(business_plan)

print("\n✅ Business plan exported to: Business_Plan_SaaS_Photo_Platform.md")
print("\n📊 Analysis complete! Review the sections above for detailed insights.")